In [15]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.pipeline import make_pipeline
import xgboost as xgb
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import GroupShuffleSplit

In [74]:
categorical_ftrs = ['Prscrbr_City',
                    'Prscrbr_State_Abrvtn',
                    'Brnd_Name',
                    'Gnrc_Name']

std_ftrs = ['Tot_Clms', 
            'Tot_30day_Fills', 
            'Tot_Day_Suply', 
            'Tot_Drug_Cst', 
            'Tot_Benes', 
            'GE65_Tot_Clms',
            'GE65_Tot_30day_Fills',
            'GE65_Tot_Drug_Cst',
            'GE65_Tot_Day_Suply',
            'GE65_Tot_Benes']



#clf = Pipeline(steps=[('preprocessor', preprocessor)])                                               


In [3]:
os.chdir('../data')

X_subsample2 = pd.read_csv('X_subsample_round2_split6.zip',compression='zip',index_col=False)
y_subsample2 = pd.read_csv('y_subsample_round2_split6.zip',compression='zip')
groups_subsample2 = pd.read_csv('groups_subsample_round2_split6.zip',compression='zip')

X_subsample2 = X_subsample2.iloc[:,1:]
y_subsample2 = y_subsample2.iloc[:,1:]
groups_subsample2 = groups_subsample2.iloc[:,1:]

y_subsample2_columns = y_subsample2.columns
#y_subsample = y_subsample.values.ravel()

In [21]:
X_subsample2

,Prscrbr_City,Prscrbr_State_Abrvtn,Brnd_Name,Gnrc_Name,Tot_Clms,Tot_30day_Fills,Tot_Day_Suply,Tot_Drug_Cst,Tot_Benes,GE65_Tot_Clms,GE65_Tot_30day_Fills,GE65_Tot_Drug_Cst,GE65_Tot_Day_Suply,GE65_Tot_Benes
0,Seattle,WA,Aripiprazole,Aripiprazole,24,24.0,718,1198.86,NaN,0.0,0.0,0.00,0.0,NaN
1,Seattle,WA,Bupropion Xl,Bupropion Hcl,21,33.0,990,538.93,NaN,NaN,NaN,NaN,NaN,NaN
2,Seattle,WA,Clonazepam,Clonazepam,27,31.0,806,284.75,NaN,27.0,31.0,284.75,806.0,NaN
3,Seattle,WA,Clonidine Hcl,Clonidine Hcl,13,17.1,514,78.13,NaN,NaN,NaN,NaN,NaN,NaN
4,Seattle,WA,Duloxetine Hcl,Duloxetine Hcl,13,13.0,390,477.59,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4626,Boca Raton,FL,Vyvanse,Lisdexamfetamine Dimesylate,18,18.0,540,5933.52,NaN,0.0,0.0,0.00,0.0,NaN
4627,Boca Raton,FL,Ziprasidone Hcl,Ziprasidone Hcl,49,59.0,1770,2619.93,NaN,NaN,NaN,NaN,NaN,NaN
4628,Boca Raton,FL,Zolpidem Tartrate,Zolpidem Tartrate,158,183.0,5490,1199.12,22.0,127.0,146.0,1057.17,4380.0,NaN
4629,Tampa,FL,Estradiol,Estradiol,17,23.0,664,1930.07,NaN,17.0,23.0,1930.07,664.0,NaN


In [78]:
param_grid = {"xgbclassifier__subsample": [0.5, 0.7, 0.9],
              "xgbclassifier__missing": [np.nan],
              "xgbclassifier__max_depth": [1, 3, 10],
              "xgbclassifier__learning_rate": [0.001, 0.01, 0.1, 0.3],
              "xgbclassifier__n_estimators": [1000],
              "xgbclassifier__gamma": [1,5,10]}


param_grid1 = {"xgbclassifier__subsample": [0.5, 0.7, 0.9],
              "xgbclassifier__missing": [np.nan],
              "xgbclassifier__learning_rate": [0.01],
              "xgbclassifier__max_depth": [1],
              "xgbclassifier__gamma": [5],
              "xgbclassifier__n_estimators": [1000]}


fit_params = {"xgbclassifier__early_stopping_rounds": 50}

scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average = 'macro'),
           'recall': make_scorer(recall_score, average = 'macro'),
           'f1_macro': make_scorer(f1_score, average = 'macro'),
           'f1_weighted': make_scorer(f1_score, average = 'weighted')}

In [43]:
X = pd.read_csv('X_3specialties_equalWeight_subsample.zip',compression='zip', index_col=False)
y = pd.read_csv('y_3specialties_equalWeight_subsample.zip',compression='zip')
groups = pd.read_csv('groups_3specialties_equalWeight_subsample.zip',compression='zip')

X = X.iloc[:,1:]
y = y.iloc[:,1:]
groups = groups.iloc[:,1:]

y_columns = y.columns

le = LabelEncoder()
y = y.values.ravel()
y = le.fit_transform(y)
y = pd.DataFrame(y)
y.columns = y_columns

In [50]:
splitter = GroupShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for i_other,i_test in splitter.split(X, y, groups):
    X_other, y_other, groups_other = X.iloc[i_other], y.iloc[i_other], groups.iloc[i_other]
    X_test, y_test, groups_test = X.iloc[i_test], y.iloc[i_test], groups.iloc[i_test]

In [69]:
len(y)

4204

In [61]:
group_kfold = GroupKFold(n_splits=10)

for train_index, test_index in group_kfold.split(X_other, y_other, groups_other):
    X_train, y_train, groups_train = X_other.iloc[train_index], y_other.iloc[train_index], groups_other.iloc[train_index]
    X_val, y_val, groups_val = X_other.iloc[test_index], y_other.iloc[test_index], groups_other.iloc[test_index]
    print(len(y_train))
    print(len(y_val))
    print(len(y_train)+len(y_val))

2822
314
3136
2822
314
3136
2822
314
3136
2822
314
3136
2822
314
3136
2823
313
3136
2823
313
3136
2823
313
3136
2822
314
3136
2823
313
3136


In [59]:
len

<function len(obj, /)>

In [90]:

def ML_pipeline_groups_GridSearchCV(X,y,groups,random_state,n_folds):
    # create a test set based on groups
    splitter = GroupShuffleSplit(n_splits=1,test_size=0.2,random_state=random_state)
    
    for i_other,i_test in splitter.split(X, y, groups):
        X_other, y_other, groups_other = X.iloc[i_other], y.iloc[i_other], groups.iloc[i_other]
        X_test, y_test, groups_test = X.iloc[i_test], y.iloc[i_test], groups.iloc[i_test]
        

    # check the split
#     print(pd.unique(groups))
#     print(pd.unique(groups_other))
#     print(pd.unique(groups_test))
    # splitter for _other
    kf = GroupKFold(n_splits=n_folds)
    # create the pipeline: preprocessor + supervised ML method

    
    clf = xgb.XGBClassifier(num_class=3,
                                eval_metric = "mlogloss",
                                objective = "multi:softprob",
                                random_state = i, 
                                use_label_encoder = False)
    
    preprocessor = ColumnTransformer(
        transformers=[
        ('onehot', OneHotEncoder(sparse=False,handle_unknown='ignore'), categorical_ftrs),
        ('std', StandardScaler(), std_ftrs)])
    
    pipe = make_pipeline(preprocessor,clf)
    
    # the parameter(s) we want to tune
    param_grid = {"xgbclassifier__subsample": [0.5, 0.7, 0.9],
              "xgbclassifier__missing": [np.nan],
              "xgbclassifier__max_depth": [1, 3, 10],
              "xgbclassifier__learning_rate": [0.001, 0.01, 0.1, 0.3],
              "xgbclassifier__n_estimators": [800, 1000],
              "xgbclassifier__gamma": [1,5,10]}
    
    # prepare gridsearch
    #grid = GridSearchCV(pipe, param_grid=param_grid,scoring = make_scorer(accuracy_score),
                        #cv=kf, return_train_score = True)
    
    grid = GridSearchCV(pipe, 
                            param_grid=param_grid,
                            scoring = "f1_macro", #‘f1_macro’ #scorer #accuracy
                            cv=kf, 
                            return_train_score = True, 
                            n_jobs=1, 
                            verbose=10)
    
    # do kfold CV on _other
    grid_result = grid.fit(X_other, y_other, groups=groups_other)
    
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    
    y_test_pred = grid.predict(X_test)
    
    #score = accuracy_score(y_test,y_test_pred)
    
    score = f1_score(y_test,y_test_pred, average = "macro")
    
    cm = confusion_matrix(y_test, y_test_pred)
    
    class_metrics = metrics.classification_report(y_test, y_test_pred, digits=3)
    
    return grid, score, cm, class_metrics



In [ ]:
%%time

test_scores = []
best_params = []
confusion_mat = []
class_met = []

for i in range(1):
    print(f'Random State # {i}')
    print()
    
    grid, score, cm, class_metrics = ML_pipeline_groups_GridSearchCV(X, y, groups, i*42, 2)
    
    confusion_mat.append(cm)
    
    class_met.append(class_metrics)
    
    print(grid.best_params_)
    
    best_params.append(grid.best_params_)
    print()
    print('best CV score:',grid.best_score_)
    print()
    print('test score:', score)
    test_scores.append(score)
    print()
    
print('test accuracy:',np.around(np.mean(test_scores),2),'+/-',np.around(np.std(test_scores),2))

Random State # 0

Fitting 2 folds for each of 216 candidates, totalling 432 fits
[CV 1/2; 1/216] START xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.001, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.5
[CV 1/2; 1/216] END xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.001, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.5;, score=(train=0.528, test=0.220) total time=   5.2s
[CV 2/2; 1/216] START xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.001, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.5
[CV 2/2; 1/216] END xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.001, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.5;, score=(train=0.499, test=0.235) total time=   6.3s
[CV 1/2; 2/216] S

[CV 1/2; 10/216] END xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.001, xgbclassifier__max_depth=3, xgbclassifier__missing=nan, xgbclassifier__n_estimators=1000, xgbclassifier__subsample=0.5;, score=(train=0.826, test=0.485) total time=  23.8s
[CV 2/2; 10/216] START xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.001, xgbclassifier__max_depth=3, xgbclassifier__missing=nan, xgbclassifier__n_estimators=1000, xgbclassifier__subsample=0.5
[CV 2/2; 10/216] END xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.001, xgbclassifier__max_depth=3, xgbclassifier__missing=nan, xgbclassifier__n_estimators=1000, xgbclassifier__subsample=0.5;, score=(train=0.712, test=0.349) total time=  25.1s
[CV 1/2; 11/216] START xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.001, xgbclassifier__max_depth=3, xgbclassifier__missing=nan, xgbclassifier__n_estimators=1000, xgbclassifier__subsample=0.7
[CV 1/2; 11/216] END xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.001, xgbclassi

[CV 1/2; 19/216] END xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.01, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.5;, score=(train=0.848, test=0.365) total time=   6.5s
[CV 2/2; 19/216] START xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.01, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.5
[CV 2/2; 19/216] END xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.01, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.5;, score=(train=0.803, test=0.350) total time=   7.0s
[CV 1/2; 20/216] START xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.01, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.7
[CV 1/2; 20/216] END xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.01, xgbclassifier__max

[CV 2/2; 28/216] END xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.01, xgbclassifier__max_depth=3, xgbclassifier__missing=nan, xgbclassifier__n_estimators=1000, xgbclassifier__subsample=0.5;, score=(train=0.996, test=0.376) total time= 1.3min
[CV 1/2; 29/216] START xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.01, xgbclassifier__max_depth=3, xgbclassifier__missing=nan, xgbclassifier__n_estimators=1000, xgbclassifier__subsample=0.7
[CV 1/2; 29/216] END xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.01, xgbclassifier__max_depth=3, xgbclassifier__missing=nan, xgbclassifier__n_estimators=1000, xgbclassifier__subsample=0.7;, score=(train=0.983, test=0.551) total time= 1.4min
[CV 2/2; 29/216] START xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.01, xgbclassifier__max_depth=3, xgbclassifier__missing=nan, xgbclassifier__n_estimators=1000, xgbclassifier__subsample=0.7
[CV 2/2; 29/216] END xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.01, xgbclassifier_

[CV 1/2; 38/216] END xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.1, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.7;, score=(train=0.994, test=0.456) total time=   9.7s
[CV 2/2; 38/216] START xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.1, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.7
[CV 2/2; 38/216] END xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.1, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.7;, score=(train=0.999, test=0.269) total time=   9.7s
[CV 1/2; 39/216] START xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.1, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.9
[CV 1/2; 39/216] END xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.1, xgbclassifier__max_dept

...

[CV 1/2; 57/216] END xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.3, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.9;, score=(train=0.994, test=0.430) total time=  19.5s
[CV 2/2; 57/216] START xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.3, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.9
[CV 2/2; 57/216] END xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.3, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.9;, score=(train=0.999, test=0.236) total time=  19.2s
[CV 1/2; 58/216] START xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.3, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=1000, xgbclassifier__subsample=0.5
[CV 1/2; 58/216] END xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.3, xgbclassifier__max_dep

[CV 2/2; 66/216] END xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.3, xgbclassifier__max_depth=3, xgbclassifier__missing=nan, xgbclassifier__n_estimators=1000, xgbclassifier__subsample=0.9;, score=(train=0.999, test=0.277) total time=  42.1s
[CV 1/2; 67/216] START xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.3, xgbclassifier__max_depth=10, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.5
[CV 1/2; 67/216] END xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.3, xgbclassifier__max_depth=10, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.5;, score=(train=0.997, test=0.478) total time= 1.4min
[CV 2/2; 67/216] START xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.3, xgbclassifier__max_depth=10, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.5
[CV 2/2; 67/216] END xgbclassifier__gamma=1, xgbclassifier__learning_rate=0.3, xgbclassifier__max_

[CV 1/2; 76/216] END xgbclassifier__gamma=5, xgbclassifier__learning_rate=0.001, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=1000, xgbclassifier__subsample=0.5;, score=(train=0.542, test=0.229) total time=   8.7s
[CV 2/2; 76/216] START xgbclassifier__gamma=5, xgbclassifier__learning_rate=0.001, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=1000, xgbclassifier__subsample=0.5
[CV 2/2; 76/216] END xgbclassifier__gamma=5, xgbclassifier__learning_rate=0.001, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=1000, xgbclassifier__subsample=0.5;, score=(train=0.503, test=0.236) total time=   9.2s
[CV 1/2; 77/216] START xgbclassifier__gamma=5, xgbclassifier__learning_rate=0.001, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=1000, xgbclassifier__subsample=0.7
[CV 1/2; 77/216] END xgbclassifier__gamma=5, xgbclassifier__learning_rate=0.001, xgbclassi

[CV 2/2; 85/216] END xgbclassifier__gamma=5, xgbclassifier__learning_rate=0.001, xgbclassifier__max_depth=10, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.5;, score=(train=0.984, test=0.344) total time= 1.4min
[CV 1/2; 86/216] START xgbclassifier__gamma=5, xgbclassifier__learning_rate=0.001, xgbclassifier__max_depth=10, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.7
[CV 1/2; 86/216] END xgbclassifier__gamma=5, xgbclassifier__learning_rate=0.001, xgbclassifier__max_depth=10, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.7;, score=(train=0.972, test=0.531) total time= 1.9min
[CV 2/2; 86/216] START xgbclassifier__gamma=5, xgbclassifier__learning_rate=0.001, xgbclassifier__max_depth=10, xgbclassifier__missing=nan, xgbclassifier__n_estimators=800, xgbclassifier__subsample=0.7
[CV 2/2; 86/216] END xgbclassifier__gamma=5, xgbclassifier__learning_rate=0.001, xgbclassi

[CV 1/2; 95/216] END xgbclassifier__gamma=5, xgbclassifier__learning_rate=0.01, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=1000, xgbclassifier__subsample=0.7;, score=(train=0.897, test=0.414) total time=  12.4s
[CV 2/2; 95/216] START xgbclassifier__gamma=5, xgbclassifier__learning_rate=0.01, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=1000, xgbclassifier__subsample=0.7
[CV 2/2; 95/216] END xgbclassifier__gamma=5, xgbclassifier__learning_rate=0.01, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=1000, xgbclassifier__subsample=0.7;, score=(train=0.889, test=0.373) total time=  13.7s
[CV 1/2; 96/216] START xgbclassifier__gamma=5, xgbclassifier__learning_rate=0.01, xgbclassifier__max_depth=1, xgbclassifier__missing=nan, xgbclassifier__n_estimators=1000, xgbclassifier__subsample=0.9
[CV 1/2; 96/216] END xgbclassifier__gamma=5, xgbclassifier__learning_rate=0.01, xgbclassifier_

In [ ]:

def ML_pipeline_XGBoost_ParamGrid(X, y, groups, i):
    # create a test set based on groups
    splitter = GroupShuffleSplit(n_splits=1,test_size=0.2,random_state=i)
    
    # Get Test Set
    for i_other,i_test in splitter.split(X, y, groups):
        X_other, y_other, groups_other = X.iloc[i_other], y.iloc[i_other], groups.iloc[i_other]
        X_test, y_test, groups_test = X.iloc[i_test], y.iloc[i_test], groups.iloc[i_test]
    
    # Get Validation Set
    
    kf = GroupKFold(n_splits=5)
    counter = 0
    for i_train, i_test in kf.split(X_other, y_other, groups_other):
        X_train, y_train, groups_train = X_other.iloc[i_train], y_other.iloc[i_train], groups_other.iloc[i_train]
        X_val, y_val, groups_val = X_other.iloc[i_test], y_other.iloc[i_test], groups_other.iloc[i_test]
        
        #print(len(y_val))
        #print(len(y_train))
        counter = counter + 1
        
        print(f"CV # {counter}")
        X_prep = preprocessor.fit_transform(X_train)
        # collect feature names
        
        feature_names = preprocessor.get_feature_names_out()
        
        df_train = pd.DataFrame(data=X_prep,columns=feature_names)
        print(f"Train Set Shape (after preprocessing): {df_train.shape}")
        print()
        
        # transform the CV
        df_CV = preprocessor.transform(X_val)
        df_CV = pd.DataFrame(data=df_CV,columns = feature_names)
        print(f"CV Set Shape (after preprocessing): {df_CV.shape}")
        print()
        # transform the test
        df_test = preprocessor.transform(X_test)
        df_test = pd.DataFrame(data=df_test,columns = feature_names)
        print(f"Test Set Shape (after preprocessing): {df_test.shape}")
        print()
        
        y_CV = y_val
        
        total_accuracy, f1, cm = reduced_feature_xgb(df_train, y_train, df_CV, y_CV, df_test, y_test, i)
        
        return total_accuracy, f1, cm

In [93]:
"""


%%time

## Perform 60-20-20 Split
# splitter for subsampled data
stratGroupKFold2 = StratifiedGroupKFold(n_splits=5)

# splitter for other
stratGroupKFold3 = StratifiedGroupKFold(n_splits=4)

# label encoder for XGBoost
le = LabelEncoder()
y_subsample2 = le.fit_transform(y_subsample2)

# Initialize lists
nr_states = [0,1,2] # ,1,2
test_scores = []
final_models = []
best_params = []
test_scores = []
counter = 0

# num of lists should equal number of test sets that the StratifiedGroupKFold will generate
# each list should contain entries with the same number of random state
    
list1_params = []
list1_grid = []
list1_pred = []
list1_score = []
list1_means = []
list1_cm = []
list1_stds = []
list1_metrics = []

list1_f1 = []

list2_params = []
list2_grid = []
list2_pred = []
list2_score = []
list2_means = []
list2_stds = []
list2_cm = []
list2_ytest = []
list2_metrics = []
list2_f1 = []

list3_params = []
list3_grid = []
list3_pred = []
list3_score = []
list3_means = []
list3_stds = []
list3_cm = []
list3_metrics = []
list3_f1 = []

list4_params = []
list4_grid = []
list4_pred = []
list4_score = []
list4_means = []
list4_stds = []
list4_cm = []
list4_metrics = []
list4_f1 = []

list5_params = []
list5_grid = []
list5_pred = []
list5_score = []
list5_means = []
list5_stds = []
list5_cm = []
list5_metrics = []
list5_f1 = []

for i_other2,i_test2 in stratGroupKFold2.split(X_subsample2.values, y_subsample2, groups_subsample2.values):
    
    counter = counter + 1
    
    X_other2, y_other2, groups_other2 = X_subsample2.values[i_other2], y_subsample2[i_other2], groups_subsample2.values[i_other2]
    X_test2, y_test2, groups_test2 = X_subsample2.values[i_test2], y_subsample2[i_test2], groups_subsample2.values[i_test2]

    # Reshape the data
    
    X_other2 = pd.DataFrame(X_other2)
    X_other2.columns = X_subsample2.columns
    
    X_test2 = pd.DataFrame(X_test2)
    X_test2.columns = X_subsample2.columns

    y_other2 = pd.DataFrame(y_other2)
    y_other2.columns = y_subsample2_columns
    

    groups_other2 = pd.DataFrame(groups_other2)
    groups_other2.columns = groups_subsample2.columns

    y_other2 = np.reshape(np.array(y_other2), (1, -1)).ravel()
    
    
    y_test2 = np.reshape(np.array(y_test2), (1, -1)).ravel()
    
    
    
    print(f'Test Set #{counter}')

    print("    Test Set Size:", len(y_test2))
    
    print("    X_test Shape:", X_test2.shape)

    print()
    
    for i in range(len(nr_states)):

        print("         Random State:", i)
        print()

        # Perform n-Fold CV
        cv = stratGroupKFold3.split(X_other2, y_other2, groups_other2)


        # Initialize XGBoost Classifier
        clf = xgb.XGBClassifier(num_class=3,
                                eval_metric = "mlogloss",
                                objective = "multi:softprob",
                                random_state = i, 
                                use_label_encoder = False)

        pipe = make_pipeline(preprocessor,clf)

        grid = GridSearchCV(pipe, 
                            param_grid=param_grid,
                            scoring = "f1_macro", #‘f1_macro’ #scorer #accuracy
                            cv=cv, 
                            return_train_score = True, 
                            n_jobs=1, 
                            verbose=10)
        
        # Compute sample weights
        weights = compute_sample_weight(class_weight='balanced', y = y_other2)
        
        print(f'Feature weights: {np.unique(weights)}')

        grid_result = grid.fit(X_other2, y_other2, groups = groups_other2,
                              xgbclassifier__sample_weight = weights) #xgbclassifier__early_stopping_rounds=50
        
        print()
        print('best model parameters:', grid.best_params_)

        print()

        print('validation score:',grid.best_score_)
        
        print()
        
        means = grid_result.cv_results_['mean_test_score']
        stds = grid_result.cv_results_['std_test_score']
        
        y_test_pred = grid.predict(X_test2)
        score = accuracy_score(y_test2,y_test_pred)
        cm = confusion_matrix(y_test2, y_test_pred)
        f1 = f1_score(y_test2, y_test_pred)
        class_metrics = metrics.classification_report(y_test2, y_test_pred, digits=3)
        
        print('Test (accuracy) score:', score)
        
        if counter == 1:
            
            list1_params.append(grid.best_params_)

            list1_grid.append(grid)

            list1_pred.append(y_test_pred)
            
            list1_score.append(score)
            
            list1_cm.append(cm)
            
            list1_f1.append(f1)
            
            list1_metrics.append(class_metrics)

        if counter == 2:
            list2_params.append(grid.best_params_)

            list2_grid.append(grid)
            
            list2_pred.append(y_test_pred)
            
            list2_score.append(score)
            
            list2_cm.append(cm)
            
            list2_f1.append(f1)
            
            list2_metrics.append(class_metrics)
            
        if counter == 3:
            list3_params.append(grid.best_params_)

            list3_grid.append(grid)

            list3_pred.append(y_test_pred)

            list3_score.append(score)
            
            list3_cm.append(cm)
            
            list3_f1.append(f1)
            
            list3_metrics.append(class_metrics)
            
        if counter == 4:
            list4_params.append(grid.best_params_)

            list4_grid.append(grid)

            list4_pred.append(y_test_pred)

            list4_score.append(score)
            
            list4_cm.append(cm)
            
            list4_f1.append(f1)
            
            list4_metrics.append(class_metrics)
            
        if counter == 5:
            list5_params.append(grid.best_params_)

            list5_grid.append(grid)

            list5_pred.append(y_test_pred)

            list5_score.append(score)
            
            list5_cm.append(cm)
            
            list5_f1.append(f1)
            
            list5_metrics.append(class_metrics)
            
        print()    
    
    
"""

'\n\n\n%%time\n\n## Perform 60-20-20 Split\n# splitter for subsampled data\nstratGroupKFold2 = StratifiedGroupKFold(n_splits=5)\n\n# splitter for other\nstratGroupKFold3 = StratifiedGroupKFold(n_splits=4)\n\n# label encoder for XGBoost\nle = LabelEncoder()\ny_subsample2 = le.fit_transform(y_subsample2)\n\n# Initialize lists\nnr_states = [0,1,2] # ,1,2\ntest_scores = []\nfinal_models = []\nbest_params = []\ntest_scores = []\ncounter = 0\n\n# num of lists should equal number of test sets that the StratifiedGroupKFold will generate\n# each list should contain entries with the same number of random state\n    \nlist1_params = []\nlist1_grid = []\nlist1_pred = []\nlist1_score = []\nlist1_means = []\nlist1_cm = []\nlist1_stds = []\nlist1_metrics = []\n\nlist1_f1 = []\n\nlist2_params = []\nlist2_grid = []\nlist2_pred = []\nlist2_score = []\nlist2_means = []\nlist2_stds = []\nlist2_cm = []\nlist2_ytest = []\nlist2_metrics = []\nlist2_f1 = []\n\nlist3_params = []\nlist3_grid = []\nlist3_pred = 

In [13]:
list3_cm

[]

ValueError: Specifying the columns using strings is only supported for pandas DataFrames

Confusion Matrix after performing XGBoost with gridsearchCV and multiple random states, 

35 min for paramgrid1 with 60-20-20

### Save XGBoost Results to a txt file

In [ ]:
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))